In [1]:
import logging
from typing import Optional, List
import partitura as pt
import numpy as np
import os

os.chdir(r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo")

from sms.src.log import configure_logging
from sms.src.synthetic_data.utils import midi_to_note_array

logger = logging.getLogger(__name__)
configure_logging()

In [2]:
def midi_to_all_bars(
    midi_path: str,
    rest_pitch: float = -1,
    remove_rests: bool = True
) -> List[np.ndarray]:
    """
    Extracts all bars from a MIDI file and returns them as a list of note arrays.

    Args:
        midi_path (str): Path to the MIDI file.
        rest_pitch (float): Pitch value for rests.
        remove_rests (bool): Whether to remove rests from the note arrays.

    Returns:
        List[np.ndarray]: List of arrays, each representing a bar with columns [duration_beat, pitch].
    """
    score = pt.load_score(midi_path)

    if not score.parts:
        raise ValueError("No parts found in the score.")
    
    part = score.parts[0]
    note_arr = part.note_array()
    
    last_note = note_arr[-1]
    last_note_end = last_note['onset_beat'] + last_note['duration_beat']
    total_bars = int(np.ceil(last_note_end / 4))  # Assuming 4 beats per bar
    print(f"Total bars: {total_bars}")
    all_bars = []
    for bar in range(total_bars):
        bar_notes = midi_to_note_array(
            midi_path=midi_path,
            num_bars=1,
            start_bar=bar,
            rest_pitch=rest_pitch,
            remove_rests=remove_rests
        )
        all_bars.append(bar_notes)

    return all_bars

path = r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\synthetic_dataset\monophonic_midis\maestro\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav_mono.mid"
all_bars = midi_to_all_bars(path)


c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:524: UserWarning: pitch spelling
  warnings.warn("pitch spelling")
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\utils\misc.py:68: UserWarning: voice estimation
  return f(*args, **kwargs)
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: create_part
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: add notes
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: add time sigs and measures
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: tie notes
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: find tuplets
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\sit

Total bars: 90


[2024-10-02 16:09:05] [INFO ] Extracted notes: [(2.  , 1.25, 2.  , 1.25, 440, 275, 42, 1, 'n0')
 (3.25, 0.75, 3.25, 0.75, 715, 165, 49, 1, 'n1')]. Returning None.
[2024-10-02 16:09:08] [WARNING] Total duration 3.75 does not match expected duration 4
[2024-10-02 16:09:08] [INFO ] Adjusted last note/rest duration by 0.25 to match expected duration
[2024-10-02 16:09:08] [WARNING] Total duration 3.75 does not match expected duration 4
[2024-10-02 16:09:08] [INFO ] Adjusted last note/rest duration by 0.25 to match expected duration
[2024-10-02 16:09:13] [WARNING] Total duration 3.75 does not match expected duration 4
[2024-10-02 16:09:13] [INFO ] Adjusted last note/rest duration by 0.25 to match expected duration
[2024-10-02 16:09:16] [WARNING] Total duration 3.75 does not match expected duration 4
[2024-10-02 16:09:16] [INFO ] Adjusted last note/rest duration by 0.25 to match expected duration
[2024-10-02 16:09:18] [WARNING] Total duration 3.75 does not match expected duration 4
[2024-10-0

In [6]:
def midi_to_all_bars_efficient(
    midi_path: str,
    rest_pitch: float = -1,
    remove_rests: bool = True
) -> List[np.ndarray]:
    """
    Efficiently extracts all bars from a MIDI file and returns them as a list of note arrays.

    Args:
        midi_path (str): Path to the MIDI file.
        rest_pitch (float): Pitch value for rests.
        remove_rests (bool): Whether to remove rests from the note arrays.

    Returns:
        List[np.ndarray]: List of arrays, each representing a bar with columns [duration_beat, pitch].
    """
    score = pt.load_score(midi_path)

    if not score.parts:
        raise ValueError("No parts found in the score.")
    
    part = score.parts[0]
    note_arr = part.note_array()
    
    last_note = note_arr[-1]
    last_note_end = last_note['onset_beat'] + last_note['duration_beat']
    total_bars = int(np.ceil(last_note_end / 4))  # Assuming 4 beats per bar

    all_bars = []
    beats_per_bar = 4

    for bar in range(total_bars):
        start_beats = bar * beats_per_bar
        end_beats = start_beats + beats_per_bar

        # Filter notes for this bar
        bar_notes = note_arr[
            (note_arr['onset_beat'] < end_beats) &
            (note_arr['onset_beat'] + note_arr['duration_beat'] > start_beats)
        ]

        if len(bar_notes) <= 3:
            logger.info(f"Bar {bar} has 3 or fewer notes. Skipping.")
            continue

        # Sort notes by onset_beat
        bar_notes = bar_notes[np.argsort(bar_notes['onset_beat'])]

        # Process notes for this bar
        duration_pitch = []
        previous_end = start_beats

        for note in bar_notes:
            note_onset = note['onset_beat']
            note_duration = note['duration_beat']
            note_pitch = note['pitch']

            adjusted_onset = max(note_onset, start_beats)
            actual_duration = min(note_onset + note_duration, end_beats) - adjusted_onset

            if adjusted_onset > previous_end:
                rest_duration = adjusted_onset - previous_end
                if remove_rests:
                    actual_duration += rest_duration
                else:
                    duration_pitch.append([rest_duration, rest_pitch])

            duration_pitch.append([actual_duration, note_pitch])
            previous_end = adjusted_onset + actual_duration

        # Handle end rest
        if previous_end < end_beats:
            rest_duration = end_beats - previous_end
            if remove_rests:
                duration_pitch[-1][0] += rest_duration
            else:
                duration_pitch.append([rest_duration, rest_pitch])

        # Convert to numpy array and round durations
        bar_array = np.array(duration_pitch, dtype=float)
        bar_array[:, 0] = np.round(bar_array[:, 0], decimals=3)

        # Validate total duration
        total_duration = np.sum(bar_array[:, 0])
        if not np.isclose(total_duration, beats_per_bar, atol=1e-8):
            logger.warning(f"Bar {bar}: Total duration {total_duration} does not match expected duration {beats_per_bar}")
            duration_diff = beats_per_bar - total_duration
            bar_array[-1, 0] += duration_diff
            logger.info(f"Adjusted last note/rest duration by {duration_diff} to match expected duration")

        all_bars.append(bar_array)

    return all_bars

path = r"C:\Users\cunn2\OneDrive\DSML\Project\thesis-repo\data\synthetic_dataset\monophonic_midis\maestro\MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_01_Track01_wav_mono.mid"
all_bars = midi_to_all_bars_efficient(path)

c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:524: UserWarning: pitch spelling
  warnings.warn("pitch spelling")
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\utils\misc.py:68: UserWarning: voice estimation
  return f(*args, **kwargs)
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: create_part
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: add notes
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: add time sigs and measures
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: tie notes
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\site-packages\partitura\io\importmidi.py:607: UserWarning: find tuplets
  part = create_part(
c:\Users\cunn2\miniconda3\envs\sms\Lib\sit

In [8]:
all_bars

def test_bar_durations(bar_list: List[np.ndarray], expected_duration: float = 4.0, tolerance: float = 1e-6) -> bool:
    """
    Test if the sum of durations in each bar is equal to the expected duration.

    Args:
        bar_list (List[np.ndarray]): List of numpy arrays representing bars.
        expected_duration (float): Expected total duration of each bar (default is 4.0).
        tolerance (float): Tolerance for floating point comparison (default is 1e-6).

    Returns:
        bool: True if all bars have the correct total duration, False otherwise.
    """
    for i, bar in enumerate(bar_list):
        total_duration = np.sum(bar[:, 0])
        if not np.isclose(total_duration, expected_duration, atol=tolerance):
            print(f"Bar {i} has incorrect duration: {total_duration} (expected {expected_duration})")
            return False
    return True

test_bar_durations(all_bars)


True

In [13]:
from sms.src.synthetic_data.note_arr_mod import NoteArrayModifier, NoteArrayModifierSettings

settings = NoteArrayModifierSettings(
    transposition_semitone_range=(-4, 4),
    notes_to_pitch_shift=1,
    note_pitch_shift_range=(-4, 4),
    notes_to_scale=1,
    note_duration_scale_options=(0.5, 1.5, 2),
    notes_to_delete=1,
    notes_to_insert=1,
    insert_note_duration_options=(0.25, 0.5),
    insert_note_relative_pitch_range=(-4, 4)
)

aug_dict = {
    "use_transposition": True,
    "use_shift_selected_notes_pitch": True,
    "use_change_note_durations": True,
    "use_delete_notes": True,
    "use_insert_notes": True
}

mod = NoteArrayModifier(settings=settings)
for bar in all_bars:
    mod(bar, aug_dict)



[2024-10-02 16:26:55] [DEBUG] Transposing non-rest notes by 1 semitones.
[2024-10-02 16:26:55] [DEBUG] Shifting note at index 3 by -6 semitones.
[2024-10-02 16:26:55] [DEBUG] Scaling duration of note at index 4 by a factor of 2.0.
[2024-10-02 16:26:55] [DEBUG] Inserting note at index 1 with duration 0.25 and relative pitch 4.
[2024-10-02 16:26:55] [DEBUG] Deleting notes at indices [4].
[2024-10-02 16:26:55] [DEBUG] Removed note 8 with duration 0.5 to adjust total duration.
[2024-10-02 16:26:55] [DEBUG] Removed note 7 with duration 0.25 to adjust total duration.
[2024-10-02 16:26:55] [DEBUG] Transposing non-rest notes by 2 semitones.
[2024-10-02 16:26:55] [DEBUG] Shifting note at index 6 by 5 semitones.
[2024-10-02 16:26:55] [DEBUG] Scaling duration of note at index 6 by a factor of 2.0.
[2024-10-02 16:26:55] [DEBUG] Inserting note at index 0 with duration 0.25 and relative pitch 1.
[2024-10-02 16:26:55] [DEBUG] Deleting notes at indices [8].
[2024-10-02 16:26:55] [DEBUG] Removed note 9